# NFL Fantasy Prediction Engine - Google Colab Training

This notebook trains the multi-model prediction system using your feature database.

## 1. Setup Environment

In [ ]:
# Install required packages
!pip install -q scikit-learn xgboost lightgbm pandas numpy

In [ ]:
# Mount Google Drive (if using Drive for database storage)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# OR: Upload database directly
# from google.colab import files
# uploaded = files.upload()  # Select nfl_data.db

In [ ]:
# Clone the repo (if not already done)
!git clone https://github.com/netprocesssolutions/nfl_prediction_engine.git
%cd nfl_prediction_engine

## 2. Verify Database

In [ ]:
import sqlite3
import pandas as pd

# Connect to database
DB_PATH = 'phase_1/database/nfl_data.db'
conn = sqlite3.connect(DB_PATH)

# Check database stats
cur = conn.cursor()
cur.execute('SELECT COUNT(*) FROM player_game_features')
rows = cur.fetchone()[0]
cur.execute('SELECT COUNT(*) FROM pragma_table_info("player_game_features")')
cols = cur.fetchone()[0]

print(f'Database loaded: {rows:,} rows, {cols} columns')

# Check for PBP features
cur.execute('SELECT name FROM pragma_table_info("player_game_features") WHERE name LIKE "pbp_%"')
pbp_cols = len(cur.fetchall())
print(f'PBP feature columns: {pbp_cols}')

conn.close()

## 3. Train Models

In [ ]:
# Quick training (for testing)
!python -m phase_4.training.train --seasons 2021 2022 2023 --val-season 2023 --test-season 2024 --quick

In [ ]:
# Full training (better results, takes longer)
# !python -m phase_4.training.train --seasons 2021 2022 2023 --val-season 2023 --test-season 2024

## 4. Generate Predictions

In [ ]:
# Predict for a specific week
!python -m phase_4.predict --season 2024 --week 14

In [ ]:
# Export predictions to CSV
!python -m phase_4.predict --season 2024 --week 14 --output predictions_week14.csv

In [ ]:
# View predictions
import pandas as pd
preds = pd.read_csv('predictions_week14.csv')
print(f'Top 20 Predictions (PPR):')
preds[['player_name', 'position', 'team', 'opponent', 'pred_fp_ppr']].head(20)

## 5. Download Results

In [ ]:
# Download predictions CSV
from google.colab import files
files.download('predictions_week14.csv')

In [ ]:
# Download trained models (optional)
import shutil
shutil.make_archive('trained_models', 'zip', 'phase_4/saved_models')
files.download('trained_models.zip')